In [ ]:
import ipywidgets as widgets
from IPython.display import Latex
import re

def addConstraint(b):
    st.append(widgets.Text(placeholder="Vincolo {}".format(len(st)+1)))
    with output:
        display(st[-1])

of:widgets.Text = widgets.Text(placeholder="Funzione obiettivo",description="min")
output:widgets.Output = widgets.Output()
st:list[widgets.Text] = []
addSt:widgets.Button = widgets.Button(description="Aggiungi vincolo")
addSt.on_click(addConstraint)
display(widgets.Label(value="Inserisci funzione obiettivo in forma di minimo e vincoli, si assumono tutte le variabili positive"))
display(widgets.Label(value="Tutte le equazioni vanno scritte secondo la notazione Latex, ad esempio:"))
display(Latex("3x_1+\\frac{2}{5}x_2=5 diventa $3x_1+\\frac{2}{5}x_2=5$"))
display(widgets.Label(value="Tutte le variabili si devono chiamare x_n con n numero intero"))
display(of)
display(widgets.Label(value="Premi per inserire un nuovo vincolo, si accettano solo problemi in forma standard"))
display(addSt, output)

La cella di sotto è per controllare se l'input è corretto

In [ ]:
output:str = "$\\min" + of.value + "$<br>s.t.<br>"
for i in st:
    output +="$"+ i.value + "$<br>"
display(Latex(output))

Eseguire questa cella almeno una volta prima di quella sotto

In [ ]:
var_pattern:str = re.compile("(?:[+]?(-?[^x]*)x_([^-+=]*))")
def parse_variables_of(f:str):
    variables:list[tuple[int,str]] = []

    find:re.Match = var_pattern.search(f)
    while (find is not None):
        variables.append((int(find.groups(0)[1]), find.groups(0)[0]))
        find = var_pattern.search(f, pos=find.end())

    max:int = 0
    for i in variables: 
        if i[0] > max: max = i[0]
    out:list[str] = [None] * max
    for i in variables:
        out[i[0]-1] = i[1]
    for i in range(len(out)):
        if out[i] is None: out[i] = 0
    return out

def parse_variables_st(s:list[widgets.Text]):
    variables:list[list[tuple[int,str]]] = []
    kv_pattern = re.compile(".*=(?P<kv>.*?)$")

    for i in s:
        temp:list[tuple[int, str]] = []
        find:re.Match = var_pattern.search(i.value)
        while (find is not None):
            t:str = find.groups(0)[0]
            if t=='': t = 1
            elif t=='-': t ="-1"
            temp.append((int(find.groups(0)[1]), t))
            find = var_pattern.search(i.value, pos=find.end())
        variables.append(temp)
    
    max:int = 0
    for i in variables:
        for j in i:
            if j[0] > max: max = j[0]
    out:list[list[str]] = [[None for _ in range(max+1)] for _ in range(len(s))]
    for i in range(len(out)): out[i][max] = int(kv_pattern.match(s[i].value).group("kv"))
    for i in range(len(variables)):
        for j in range(len(variables[i])):
            out[i][variables[i][j][0]-1] = variables[i][j][1]
    for i in range(len(out)):
        for j in range(len(out[i])):
            if out[i][j] is None: out[i][j] = 0
    return out

def output_tableau(t:list[list[str]]):
    out:str = "<table>"
    for i in t:
        out += "<tr>"
        for j in i:
            out += "<td>\n\n$" + repr(j).replace("'",'').replace("\\\\",'\\') + "$\n</td>\n"
        out += "</tr>"
    out += "</table>"
    display(Latex(out))

Questa cella risolve il problema

In [ ]:
for i in st:
    if '\\geq' in i.value:
        raise Exception("Problema non in forma standard")

first_row:list[str] = []
other_rows:list[list[str]] = []
first_row = parse_variables_of(of.value)
other_rows = parse_variables_st(st)

lf:int = len(first_row)
lo:int = len(other_rows[0]) - 1
if lf > lo:
    first_row.append(0)
    for i in range(len(other_rows)):
        for _ in range(lo, lf):
            other_rows[i].append(0)
        other_rows[i][-1] = other_rows[i][lo]
        other_rows[i][lo] = 0
else:
    for _ in range(lf, lo+1): first_row.append(0)

tableau:list[list[str]] = []
tableau.append(first_row)
for row in other_rows: tableau.append(row)

output_tableau(tableau)